### Project Milestone P2

The following notebook contains the first data manipulations and analysis. 
The used datasets comme from two websites: BeerAdvocate and RateBeer. We also have the data present in the two datasets. 
For each website, the used files are:   
- beers.csv 
- breweries.csv
- users.csv
- ratings.txt
- reviews.txt

In order to be able to use the ratings, the text file was converted into csv using the script txt-to-csv.py. 

We firstly combine the data from the two datasets, while taking into consideration the data present in both datasets. 

### PART 1: Merging (instead of matching) the data from the two websites into one.

In [1]:
import pandas as pd
import numpy as np
from helpers import *
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.ticker import MaxNLocator
from geopy.geocoders import Nominatim
import folium
from folium.plugins import HeatMap
from IPython.display import display
from geopy.distance import geodesic
import math
from statsmodels.stats import diagnostic
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
%load_ext autoreload
%autoreload 2

1. Loading the datasets

In [2]:
beers_rb = pd.read_csv("dataset/RateBeer/beers.csv")
beers_ba = pd.read_csv("dataset/BeerAdvocate/beers.csv")
beers_matched = pd.read_csv("dataset/MatchedData/beers.csv")


KeyboardInterrupt



In [ ]:
users_rb = pd.read_csv("dataset/RateBeer/users.csv")
users_ba = pd.read_csv("dataset/BeerAdvocate/users.csv")
users_matched = pd.read_csv("dataset/MatchedData/users.csv")

In [ ]:
ratings_rb = pd.read_csv("dataset/RateBeer/ratings-reviews.csv")
ratings_ba = pd.read_csv("dataset/BeerAdvocate/ratings.csv")
ratings_matched = pd.read_csv("dataset/MatchedData/ratings.csv")

In [ ]:
brewery_rb = pd.read_csv("dataset/RateBeer/breweries.csv")
brewery_ba = pd.read_csv("dataset/BeerAdvocate/breweries.csv")
brewery_matched = pd.read_csv("dataset/MatchedData/breweries.csv")

2. Changing the beer_id in the BeerAdvocate dataset, so that each beer id is unique

In [ ]:
# ID's of the beers coming from the two websites should be unique. However, there are intersections for now, which would cause problems after merging. That's why we add 1000000 to id's of the BeerAdvocate beers.

ba_id = beers_ba['beer_id'].values
rb_id = beers_rb['beer_id'].values
ba_id = ba_id + 1000000

#Checking whether the overlapping problem is solved:
if np.intersect1d(ba_id, rb_id).size > 0:
    print(np.intersect1d(ba_id, rb_id).size)
    
beers_ba['beer_id'] = beers_ba['beer_id'] + 1000000
ratings_ba['beer_id'] = ratings_ba['beer_id'] + 1000000
beers_matched['ba.5'][1:] = beers_matched['ba.5'][1:].astype(int) + 1000000
beers_matched['rb.4'][1:] = beers_matched['rb.4'][1:].astype(int)

#Finding the dictionary that maps the beers from BeerAdvocate to RateBeer (for the beers that are the same) by the help of matched data provided:
ba_to_rb_beer_id_dict = dict(zip(beers_matched['ba.5'], beers_matched['rb.4']))

3. Changing the brewery_id in the BeerAdvocate dataset, so that each brewery id is unique

In [ ]:
# ID's of the breweries coming from the two websites should be unique. However, there are intersections for now, which would cause problems after merging. That's why we add 1000000 to id's of the BeerAdvocate breweries, again.

ba_id = brewery_ba['id'].values
rb_id = brewery_rb['id'].values
ba_id = ba_id + 1000000

#Checking whether the overlapping problem is solved:
if np.intersect1d(ba_id, rb_id).size > 0:
    print(np.intersect1d(ba_id, rb_id).size)

brewery_ba['id'] = brewery_ba['id'] + 1000000
beers_ba['brewery_id'] = beers_ba['brewery_id'] + 1000000
ratings_ba['brewery_id'] = ratings_ba['brewery_id'] + 1000000
brewery_matched['ba'][1:] = brewery_matched['ba'][1:].astype(int) + 1000000
brewery_matched['rb'][1:] = brewery_matched['rb'][1:].astype(int)

#Finding the dictionary that maps the breweries from BeerAdvocate to RateBeer (for the breweries that are the same) by the help of matched data provided:
ba_to_rb_brewery_id_dict = dict(zip(brewery_matched['ba'], brewery_matched['rb']))

4. Checking for the users id

In [ ]:
ba_id = users_ba['user_id'].values
rb_id = users_rb['user_id'].astype(str).values

#Checking if there is the same ID overlap problem for the users as well:
if np.intersect1d(ba_id, rb_id).size > 0:
    print(np.intersect1d(ba_id, rb_id).size)

#Finding the dictionary that maps the users from BeerAdvocate to RateBeer (for the users that are the same) by the help of matched data provided:  
ba_to_rb_user_id_dict = dict(zip(users_matched['ba.4'], users_matched['rb.3']))

5. Finding the style names present in each dataset, and harmonize the styles between the websites

In [ ]:
#Finding the style of beers present in the matched data and RateBeer data separately:
matched_rb_beers = beers_matched['rb.12'].unique()
all_rb_beers = beers_rb['style'].unique()

#Finding the styles that are in RateBeer but not in the matched data:
elements_not_in_matched = [element for element in all_rb_beers if element not in matched_rb_beers]

#The styles that are in RateBeer but not in the matched data have received very few ratings. Furthermore, we will map the styles of BeerAdvocate to the styles of RateBeer using the matched data. Since no BeerAdvocate beers will be mapped to this set of beers, they will appear even less popular in the results. Therefore, we eliminate these styles here.
beers_rb = beers_rb[~beers_rb['style'].isin(elements_not_in_matched)]
ratings_rb = ratings_rb[~ratings_rb['style'].isin(elements_not_in_matched)]

In [ ]:
#Inspecting the matched data we realize that there is no one to one matched between the "style" labels of the two websites. For example: a beer with style label of "English Pale Ale" on BeerAdvocate is labeled as having the style "Golden Ale/Blond Ale" in RateBeer. But another beer, whose style is also "English Pale Ale" on BeerAdvocate has this time style label of "Amber Ale" in RateBeer. 

# In order to match the BeerAdvocate styles to the RateBeer styles, we look at all the instances of each unique beer in BeerAdvocate and find its most frequent corresponding style in RateBeer. We then collect each unique style in BeerAdvocate, its most frequent match and the most frequent match count in a dataframe.

ba_to_rb = []
for style_value in beers_matched['ba.14'].unique():
    # Filter the DataFrame for rows where 'style_column1' matches the current value
    filtered_rows = beers_matched[beers_matched['ba.14'] == style_value]
    
    # Find the most frequent match in 'style_column2'
    most_frequent_match = filtered_rows['rb.12'].mode().iloc[0]
    
    # Count the occurrences of the most frequent match
    match_count = filtered_rows[filtered_rows['rb.12'] == most_frequent_match].shape[0]
    
    # Append the results to the result DataFrame
    # 
    ba_to_rb.append({'style_column1': style_value,
                                  'most_frequent_match': most_frequent_match,
                                  'match_count': match_count})
ba_to_rb.pop(0)
ba_to_rb_df = pd.DataFrame(ba_to_rb)

In [ ]:
#Printing the resulting dataFrame: 
ba_to_rb_df.head()

In [ ]:
#We see that all the styles of the BeerAdvocate are mapped to 71 different beer styles in RateBeer using the above-explained procedure. We would like to have one-to-one mapping between the styles of two websites such that not only all the styles of BeerAdvocate are mapped to a style of RateBeer, all styles of RateBeer are also mapped to a BeerAdvocate style.

#For this, we first find which styles of RateBeer are not mapped to a BeerAdvocate beer in the previous cell.
rb_to_ba_styles = ba_to_rb_df.most_frequent_match.unique()
matched_rb_styles = beers_matched['rb.12'].unique()
elements_not_in_ba = [element for element in matched_rb_styles if element not in rb_to_ba_styles]

In [ ]:
#For the RateBeer styles that are not mapped, we first find their most frequent match in BeerAdvocate, and then map it back to one of the RateBeer styles as done before. By doing this, we make sure that all the BeerAdvocate styles are mapped to a RateBeer style and vice versa.

rb_to_ba_df = []
for style_value in elements_not_in_ba:
    # Filter the DataFrame for rows where 'style_column1' matches the current value
    filtered_rows = beers_matched[beers_matched['rb.12'] == style_value]
    
    # Find the most frequent match in 'style_column2'
    most_frequent_match = filtered_rows['ba.14'].mode().iloc[0]
    
    # Count the occurrences of the most frequent match
    match_count = filtered_rows[filtered_rows['ba.14'] == most_frequent_match].shape[0]
    
    # Append the results to the result DataFrame
    rb_to_ba_df.append({'style_column1': style_value,
                                  'most_frequent_match': most_frequent_match,
                                  'match_count': match_count})
rb_to_ba_df = pd.DataFrame(rb_to_ba_df)

In [ ]:
#Finding the dictionary that maps the "unmapped" RateBeer styles to one of the BeerAdvocate styles:

rb_to_ba_dict = dict(zip(rb_to_ba_df['style_column1'], rb_to_ba_df['most_frequent_match']))
beers_matched['rb.12'] = beers_matched['rb.12'].replace(rb_to_ba_dict)
beers_rb['style'] = beers_rb['style'].replace(rb_to_ba_dict)
ratings_rb['style'] = ratings_rb['style'].replace(rb_to_ba_dict)

In [ ]:
display(rb_to_ba_dict)

In [ ]:
#Mapping these styles back to one of the 71 RateBeer styles:

ba_to_rb_dict = dict(zip(ba_to_rb_df['style_column1'], ba_to_rb_df['most_frequent_match']))
beers_ba['style'] = beers_ba['style'].replace(ba_to_rb_dict)
beers_matched['rb.12'] = beers_matched['rb.12'].replace(ba_to_rb_dict)
beers_rb['style'] = beers_rb['style'].replace(ba_to_rb_dict)
ratings_rb['style'] = ratings_rb['style'].replace(ba_to_rb_dict)

In [ ]:
matched_rb_ids = beers_matched['rb.4'][1:]
matched_ba_ids = beers_matched['ba.5'][1:]

6. After those modifications, merging of the datasets

In [ ]:
#Removing the matched beers from the two websites before merging:

rb_without_matched = beers_rb[~beers_rb['beer_id'].isin(matched_rb_ids)]
ba_without_matched = beers_ba[~beers_ba['beer_id'].isin(matched_ba_ids)]

In [ ]:
#First, we take only the necessary columns of beer datasets and rename them conveniently. 

columns_to_take = ['beer_id', 'beer_name', 'brewery_id', 'brewery_name', 'style', 'abv']
column_name_mapping = {'rb.4': 'beer_id', 'rb.5': 'beer_name', 'rb.7': 'brewery_id', 'rb.8': 'brewery_name', 'rb.12': 'style', 'rb': 'abv'}
beers_matched.rename(columns = column_name_mapping, inplace = True)

#We first merge the beer datasets from the two websites. Note that the matched beers were eliminated from these datasets beforehand.
merged_beers = pd.concat([rb_without_matched[columns_to_take], ba_without_matched[columns_to_take]], axis=0, ignore_index = True)

#Now we include the matched beers in the merged datasets, making sure that the matched beers are not repeated but included only once.
merged_beers = pd.concat([merged_beers, beers_matched.iloc[1:][columns_to_take]], axis=0, ignore_index = True)

In [ ]:
#Before merging the breweries, removing the matched breweries from BeerAdvocate and then merging this directly with the original brewery data of RateBeer:

matched_ba_ids = brewery_matched['ba'][1:]
ba_without_matched = brewery_ba[~brewery_ba['id'].isin(matched_ba_ids)]
merged_breweries = pd.concat([brewery_rb, ba_without_matched], axis = 0, ignore_index = True)

In [ ]:
#Before merging the users, removing the matched users from BeerAdvocate and then merging this directly with the original user data of RateBeer:

matched_ba_ids = users_matched['ba.4'][1:]
ba_without_matched = users_ba[~users_ba['user_id'].isin(matched_ba_ids)]
merged_users = pd.concat([users_rb, ba_without_matched], axis = 0, ignore_index = True)
merged_users['user_id'] = merged_users['user_id'].astype(str)

In [ ]:
#The BeerAdvocate dataset user_id's, beer_id's and brewery_id's are changed to be aligned to the Ratebeer dataset, according to the association described earlier.  Before merging the ratings data of the two websites, we update the corresponding columns of the BeerAdvocate datasets using the dictionaries calculated beforehand. 

ratings_ba['style'] = ratings_ba['style'].replace(ba_to_rb_dict)
ratings_ba['user_id'] = ratings_ba['user_id'].replace(ba_to_rb_user_id_dict)
ratings_ba['beer_id'] = ratings_ba['beer_id'].replace(ba_to_rb_beer_id_dict)
ratings_ba['brewery_id'] = ratings_ba['brewery_id'].replace(ba_to_rb_brewery_id_dict)

In [ ]:
#Adjusting the data types:

ratings_matched = ratings_matched.iloc[1:]

ratings_matched['ba.15'] = ratings_matched['ba.15'].astype(str)
ratings_matched['ba.7'] = ratings_matched['ba.7'].astype(np.int64)
ratings_matched['ba.3'] = ratings_matched['ba.3'].astype(np.int64)

ratings_matched['rb.14'] = ratings_matched['rb.14'].astype(str)
ratings_matched['rb.7'] = ratings_matched['rb.7'].astype(np.int64)
ratings_matched['rb.3'] = ratings_matched['rb.3'].astype(np.int64)

# List of usernames present in both websites, using BA information:
listed_ba_user = list(ratings_matched['ba.15'])
listed_ba_date = list(ratings_matched['ba.7'])
listed_ba_beer = list(ratings_matched['ba.3'])

# List of usernames present in both websites, using RB information:
listed_rb_user=list(ratings_matched['rb.14'])
listed_rb_date = list(ratings_matched['rb.7'])
listed_rb_beer = list(ratings_matched['rb.3'])

# Creating triples to be able to uniquely identify the matched ratings (ratings that are given for the same beer in the two websites by the same user) in each of the websites:
couple_ba = list(zip(listed_rb_user, listed_ba_date, listed_rb_beer))
couple_rb = list(zip(listed_rb_user, listed_rb_date, listed_rb_beer))

# Selecting only the rows that do not match the triples for BeerAdvocate, dropping review column and adding a new column to indicate the website for the future z-score calculations:
ratings_ba['user_id'] = ratings_ba['user_id'].astype(str)
ratings_ba['date'] = ratings_ba['date'].astype(np.int64)
ratings_ba['beer_id'] = ratings_ba['beer_id'].astype(np.int64)
df_ba_removed = ratings_ba[~ratings_ba.set_index(['user_id', 'date', 'beer_id']).index.isin(couple_ba)]
df_ba_removed = df_ba_removed.drop('review', axis = 1)
df_ba_removed['website'] = 1

# Selecting only the rows that do not match the triples for RateBeer and adding a new column to 
# indicate the website for the future z-score calculations:
ratings_rb['user_id'] = ratings_rb['user_id'].astype(str)
ratings_rb['date'] = ratings_rb['date'].astype(np.int64)
ratings_rb['beer_id'] = ratings_rb['beer_id'].astype(np.int64)
df_rb_removed = ratings_rb[~ratings_rb.set_index(['user_id', 'date', 'beer_id']).index.isin(couple_rb)]
df_rb_removed['website'] = 0

merged_ratings = pd.concat([df_ba_removed, df_rb_removed], axis = 0, ignore_index=True)

merged_ratings['date'] = merged_ratings['date'].apply(convert_unix_timestamp)
merged_ratings['year'] = merged_ratings['date'].dt.year

In [ ]:
#Recalculating the number of ratings:

rating_counts_by_user = merged_ratings.groupby('user_id')['beer_id'].count()
rating_counts_by_user_dict = rating_counts_by_user.to_dict()
merged_users['nbr_ratings'] = merged_users['nbr_ratings'].replace(rating_counts_by_user_dict)

In [ ]:
merged_beers.to_csv("dataset/MergedData/beers.csv", index = False)
merged_breweries.to_csv("dataset/MergedData/breweries.csv", index = False)
merged_users.to_csv("dataset/MergedData/users.csv", index = False)
merged_ratings.to_csv("dataset/MergedData/ratings.csv", index = False)

In [ ]:
beers = pd.read_csv('dataset/MergedData/beers.csv')
breweries = pd.read_csv('dataset/MergedData/breweries.csv')
ratings = pd.read_csv('dataset/MergedData/ratings.csv', dtype = {'user_id': str})
users = pd.read_csv('dataset/MergedData/users.csv', dtype = {'user_id': str})

### PART 2: Preprocessing of the merged data before initial analysis

1. Preprocessing of the ratings

We compute the z-score for each rating and sub-rating feature

Two main observations were made here: 
 - The mean rating is not the same for the two websites
 - The mean rating augments with the years

Moreover, we found out that the range of data was not the same between the aspects: for example, for RateBeer, the "overall" rating goes from 1 to 20, and on BeerAdvocate, from 1 to 5. Within one website, the range of different metrics changed too. Normalizing was then very important, especially for the next steps, where correlation between each aspect and the total rating will be studied. Indeed, the range of data could influence the computation of the coefficients for the regression for example.

As we do not want those effects to be reflected on our results, the z-score was calculated. 
For each feature, we substracted the mean for this year and the corresponding website, and divided by the standard deviation.

In [ ]:
f = sns.barplot(data=ratings, x='year', y='rating', hue='website', order=sorted(ratings['year'].unique()), palette='mako')
f.tick_params(axis='x', which='major', length = 10, width=3, direction='inout')
plt.xticks(rotation=45)
f.set_title("Mean rating accross the years")

handles, labels = f.get_legend_handles_labels()
f.legend(handles, ['RateBeer', 'BeerAdvocate'], title='Website', loc='upper left')

plt.tight_layout()

In [ ]:
#Calculating z-scores for all types of ratings by normalizing with respect to the mean and standard deviation of the corresponding website's ratings of that year:

ratings['rating'] = (ratings['rating'] - ratings.groupby(['year', 'website'])['rating'].transform('mean')) / ratings.groupby(['year', 'website'])['rating'].transform('std')

ratings['appearance'] = (ratings['appearance'] - ratings.groupby(['year', 'website'])['appearance'].transform('mean')) / ratings.groupby(['year', 'website'])['appearance'].transform('std')

ratings['aroma'] = (ratings['aroma'] - ratings.groupby(['year', 'website'])['aroma'].transform('mean')) / ratings.groupby(['year', 'website'])['aroma'].transform('std')

ratings['palate'] = (ratings['palate'] - ratings.groupby(['year', 'website'])['palate'].transform('mean')) / ratings.groupby(['year', 'website'])['palate'].transform('std')

ratings['taste'] = (ratings['taste'] - ratings.groupby(['year', 'website'])['taste'].transform('mean')) / ratings.groupby(['year', 'website'])['taste'].transform('std')

ratings['overall'] = (ratings['overall'] - ratings.groupby(['year', 'website'])['overall'].transform('mean')) / ratings.groupby(['year', 'website'])['overall'].transform('std')

In [ ]:
#Visualizing the ratings dataframe as a sanity check:

ratings.head()

In [ ]:
#Inspecting some statistics of our dataset:

print(f"Number of unique beers: {len(beers['beer_id'].unique())}")
print(f"Number of beers with ratings: {len(ratings['beer_id'].unique())}\n")

print(f"Number of unique beer styles: {len(beers['style'].unique())}\n")

print(f"Number of unique breweries: {len(breweries['id'].unique())}")
print(f"Number of breweries with ratings: {len(ratings['brewery_id'].unique())}\n")

print(f"Number of unique users: {len(users['user_id'].unique())}")
print(f"Number of users with ratings: {len(ratings['user_id'].unique())}\n")

print(f"Total number of ratings: {len(ratings)}")

print("Nan: ")
ratings.isna().sum()

This is interesting to note that there is exactly the same number of missing values for the ratings of the different aspects of the beer. It is probably due to the fact that there are two "levels" of rating: an overall grade, that would be quicker to make, or a rating for each aspect (in which case the user has to respond to all the fields). 

When we will study the effect of each aspect, those NaN for those and for abv will be ignored. However, those values are not relevant for all our research, so we will keep all the values in a first time. 

The text review and usernames won't be analyzed eather, so the NaN are not problematic.

2. Preprocessing of the alcohol content

We check the values of abv, and filter the dataset.

In [ ]:
#Looking at the distribution of the alcohol by volume of beers:
sns.boxplot(beers, x = "abv")

plt.title('Distribution of Alcohol by Volume', fontsize = 12)
plt.xlabel('Alcohol by Volume', fontsize = 11)
plt.show()

We see that there are some beers with unrealisticly high alcohol percentages. After a research, we concluded that data coming from these beers are unreliable and we decided to eliminate the beers with more that 50 percent of alcohol by volume.

In [ ]:
beers = beers[beers['abv'] < 50]
ratings = ratings[ratings['abv'] < 50]

3. Check the users and their number of ratings

In [ ]:
ax = sns.histplot(users['nbr_ratings'], bins = 50, log_scale=(False, True))

ax.set_title('Histogram of Number of Ratings per User', fontsize = 12)
ax.set_xlabel('Number of Ratings', fontsize = 11)
ax.set_ylabel('Count', fontsize = 11)
plt.show()

We see that there are some users with unrealistic number of rated beers. Looking at the plot, we identify users with more than 20,000 ratings as outliers and both eliminate them as users and also eliminate their ratings.

In [ ]:
outlier_user_ids = users[users['nbr_ratings'] >= 20000]['user_id'].values
users = users[~users['user_id'].isin(outlier_user_ids)]
ratings = ratings[~ratings['user_id'].isin(outlier_user_ids)]

4. Additional datasets

In [ ]:
#Reading additional population datasets that will be useful for location filtering. The datasets are showing populations of all the relevant locations for 2010, which is a suitable year for our year range.

us_state_populations = pd.read_csv('dataset/AdditionalData/US_state_populations.csv')
world_population = pd.read_csv("dataset/AdditionalData/world_population.csv")


display(us_state_populations.head(5))
display(world_population.head(5))

In [ ]:
#Contacenating the two datasets into one and creating a location-population dictionary:

us_state_populations.rename(columns = {'states': 'location'}, inplace = True)
populations_by_location = pd.concat([us_state_populations, world_population], ignore_index = True)
populations_by_location_dict = dict(zip(populations_by_location['location'], populations_by_location['2010']))

5. Filtering locations to include in the study

To have meaningful results, we decided to remove the countries with a low number of users. In order to do that, we compute the proportion of users among the population. 

In [ ]:
#Finding the portion (as percentage) of population for each location that are users in either of the websites. Then, we print the list of locations that are not present in the additional datasets. We see that all the locations are very small countries, so that exclusion of them will not be significant for our future analysis.

num_users_by_location = users['location'].value_counts().reset_index()
percentage_users_by_location = []
missing_locations = []
for index, row in num_users_by_location.iterrows():
    location = row['location']
    if location in populations_by_location_dict:
        percentage_users_by_location.append({'location': location, 'percentage': 100 * row['count'] / populations_by_location_dict[location], 'population': populations_by_location_dict[location], 'users': row['count']})
    else:
        missing_locations.append(location)

percentage_users_by_location = pd.DataFrame(percentage_users_by_location)

print(percentage_users_by_location)  
print(missing_locations)

In [ ]:
percentage_users_by_location.sort_values(by = 'percentage', ascending = False, inplace = True)
percentage_users_by_location

In [ ]:
#Histogram of the populations

ax = sns.histplot(percentage_users_by_location['population'], bins = 50, log_scale=(True, True))
ax.set_title('Histogram of Number of Populations', fontsize = 12)
ax.set_xlabel('Population', fontsize = 11)
ax.set_ylabel('Count', fontsize = 11)
plt.show()

Looking at the histogram, we identify locations (countries) that have a population smaller than 100,000 as outliers and eliminate them.

In [ ]:
percentage_users_by_location = percentage_users_by_location[percentage_users_by_location['population'] > 1e5]

In [ ]:
print(percentage_users_by_location.to_string())

In [ ]:
#Line plot of number of ratings by year:

rating_counts_by_year = ratings['year'].value_counts().reset_index().sort_values(by = 'year')
plt.plot(rating_counts_by_year['year'], rating_counts_by_year['count'], marker = 'o')

plt.title('Histogram of Number of Populations', fontsize = 12)
plt.xlabel('Population', fontsize = 11)
plt.ylabel('Count', fontsize = 11)
plt.show()

We need to decide on a set of years for our future analysis based on the presence of sufficient amount of data. However, looking at this line plot alone is not enough to make a decision. We want to know the number of locations with more than 50 ratings for each year. 

In [ ]:
ratings_users = pd.merge(ratings, users[['user_id', 'nbr_ratings', 'location']], on='user_id',how='inner')

In [ ]:
#Creating a dataframe that gives the rating counts of each year and location pair:
year_location_counts = ratings_users.groupby(['year', 'location'])['user_id'].count().reset_index()

In [ ]:
year_location_counts_filtered = year_location_counts[year_location_counts['user_id'] >= 50]['year'].value_counts().reset_index().sort_values(by = 'year')
plt.plot(year_location_counts_filtered['year'], year_location_counts_filtered['count'], marker = 'o')


ax = plt.gca()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
plt.axhline(y = 90, color='red', linestyle='--', label = 'Horizontal Line at y = 90')
plt.title('Number of Locations with at least 50 Ratings, by Year', fontsize = 12)
plt.xlabel('Year', fontsize = 11)
plt.ylabel('# of Locations', fontsize = 11)
plt.legend()
plt.show()

We see that beginning from the year 2006, the number of locations satisfying the threshold of at least 50 ratings more or less stabilizes after crossing the number of 90 locations. Due to lack of data to conclude reliable results for the earlier years, we decide to consider only the years after and including 2006.

In [ ]:
ratings = ratings[ratings['year'] >= 2006]

In [ ]:
year_location_counts = year_location_counts[year_location_counts['user_id'] >= 50]
year_location_counts = year_location_counts[year_location_counts['year'] >= 2006]

In [ ]:
#Finding the set of locations that satisfy the at least 50 ratings limit for each year after and including 2006. These locations are identified as "valid".

valid_year_num_by_location = year_location_counts['location'].value_counts().reset_index()
valid_locations = valid_year_num_by_location[valid_year_num_by_location['count'] == 12]['location'].values
valid_locations

In [ ]:
#We decided that a sensible condition to decide the final set of locations of interest is the following: The location should be in the "valid locations" (at least 50 ratings for every year) AND [the percentage of users should be greater than or equal to 0.004 OR the number of users from that location should be at least 500]. This ensures that there are enough data in each year and it is representative of the whole population of the location.

final_locations = percentage_users_by_location[percentage_users_by_location['location'].isin(valid_locations) & ((percentage_users_by_location['percentage'] >= 0.004) | (percentage_users_by_location['users'] >= 500))]['location']

all_locations = final_locations.tolist()

final_locations

In [ ]:
# We fixed some of the brewery locations manually such as mapping individual locations in Canada into one common name "Canada" and manually searching for the brewery locations on the web that were given as website URL's. We gathered the necessary updates in a dictionary and updated the brewery locations accordingly.

brewery_name_correction_dict = {'Canada, Alberta': 'Canada', 'Canada, British Columbia': 'Canada', 'Canada, Manitoba': 'Canada', 'Canada, New Brunswick': 'Canada', 'Canada, Newfoundland and Labrador': 'Canada', 'Canada, Nova Scotia': 'Canada', 'Canada, Ontario': 'Canada', 'Canada, Quebec': 'Canada', 'Canada, Saskatchewan': 'Canada', 'Illinois</a>, 60614-4939, <a href="/place/directory/9/US/">United States': 'United States, Illinois', 'New York</a>, 13057, <a href="/place/directory/9/US/">United States': 'United States, New York', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://barleysbrewing.com" target="_blank">barleysbrewing.com': 'United States, Ohio', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://bigbuck.com" target="_blank">bigbuck.com': 'United States, Michigan', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://bjsrestaurants.com" target="_blank">bjsrestaurants.com': 'United States, California', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://bluecorncafe.com" target="_blank">bluecorncafe.com': 'United States, North Carolina', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://buckheadbrewery.com" target="_blank">buckheadbrewery.com': 'United States, Georgia', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://capcitybrew.com" target="_blank">capcitybrew.com': 'United States, Washington', 'United States, Washington DC': 'United States, Washington', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://empirebrewco.com" target="_blank">empirebrewco.com': 'United States, New York', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://eotrading.com" target="_blank">eotrading.com': 'United States, California', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://fullsailbrewing.com" target="_blank">fullsailbrewing.com': 'United States, Oregon', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://gcfb.net" target="_blank">gcfb.net': 'United States, Minnesota', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://gordonbiersch.com" target="_blank">gordonbiersch.com': 'United States, California', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://herefordandhops.com" target="_blank">herefordandhops.com': 'United States, Michigan', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://hopsonline.com" target="_blank">hopsonline.com': 'United States, New Mexico', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://ilvicino.com" target="_blank">ilvicino.com': 'United States, New Mexico', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://ironhillbrewery.com" target="_blank">ironhillbrewery.com': 'United States, Pennsylvania', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://leinie.com" target="_blank">leinie.com': 'United States, Wisconsin', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://mcmenamins.com" target="_blank">mcmenamins.com': 'United States, Oregon', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://mendobrew.com" target="_blank">mendobrew.com': 'United States, California', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://rockbottom.com" target="_blank">rockbottom.com': 'United States, Colorado', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://seadogbrewing.com" target="_blank">seadogbrewing.com': 'United States, Maine', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://sebagobrewing.com" target="_blank">sebagobrewing.com': 'United States, Maine', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://shipyard.com" target="_blank">shipyard.com': 'United States, Maine', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://snakeriverbrewing.com" target="_blank">snakeriverbrewing.com': 'United States, Wyoming', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://sonorabrew.com" target="_blank">sonorabrew.com': 'United States, California', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://steelheadbrewingco.com" target="_blank">steelheadbrewingco.com': 'United States, Oregon', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://theram.com" target="_blank">theram.com': 'United States, Oregon', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://thirstydog.com" target="_blank">thirstydog.com': 'United States, Ohio', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://tiedhouse.com" target="_blank">tiedhouse.com': 'United States, Pennsylvania', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://triumphbrew.com" target="_blank">triumphbrew.com': 'United States, New Jersey', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://tworows.com" target="_blank">tworows.com': 'United States, Texas', 'United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://weepingradish.com" target="_blank">weepingradish.com': 'United States, North Carolina', 'Utah</a><br><a href="http://utahbeers.com" target="_blank">utahbeers.com': 'United States, Utah'}

breweries['location'] = breweries['location'].replace(brewery_name_correction_dict)

In [ ]:
#Saving the filtered data:

users = users[users['location'].isin(all_locations)]
ratings = ratings[ratings['user_id'].isin(users['user_id'].unique())]

beers.to_csv("dataset/FilteredData/beers.csv", index = False)
breweries.to_csv("dataset/FilteredData/breweries.csv", index = False)
users.to_csv("dataset/FilteredData/users.csv", index = False)
ratings.to_csv("dataset/FilteredData/ratings.csv", index = False)

### PART 3: Initial Analysis

1.  Visualizations on the World Map

In [ ]:
#Using a function that returns the corresponding latitude and longitude of the input location name, we put all our final location set on the world map to visualize their geographical distribution.

def geocode_location(location_name):
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(location_name)
    if location:
        return location.latitude, location.longitude
    else:
        return None

location_latitudes_longitudes = []
location_latitudes_longitudes_tuples = []
for location in all_locations:
    res = geocode_location(location)
    if res:
        lat,long = res
        location_latitudes_longitudes.append([lat, long])
        location_latitudes_longitudes_tuples.append(res)

In [ ]:
map_center = [20,0]
my_map = folium.Map(location = map_center, zoom_start = 1)

for i, location in enumerate(all_locations):
    folium.CircleMarker(location = location_latitudes_longitudes[i], radius = 5, color='blue', fill=True, fill_color='blue', fill_opacity = 0.7, popup = location).add_to(my_map)
    
display(my_map)

One can see that our set of locations are distributed such that they are mainly populated around North America and North Europe. However, we have also some outliers that are far away from these groups, such as New Zealand or Australia. Such a contrast could allow us to investigate and compare/contrast the trend behaviour among the neighbor locations as well as more separated ones.

In [ ]:
#Using a function that returns the corresponding latitude and longitude of the input location name, we put all our brewery locations on the world map to visualize their geographical distribution.

all_brewery_locations = breweries['location'].tolist()
all_distinct_brewery_locations = list(set(all_brewery_locations))
brewery_latitudes_longitudes = []
my_map = folium.Map(location = [20, 0], zoom_start = 3)
for location in all_distinct_brewery_locations:
    try:
        res = geocode_location(location)
    except:
        continue
    if res:
        lat,long = res
        brewery_latitudes_longitudes.append([lat, long])
 
#Using a heatmap to better indicate the density of breweries per location:        
HeatMap(brewery_latitudes_longitudes).add_to(my_map)
display(my_map)

2. Popularity and Rating Trend Analysis

3.2.1: Initial Analysis of Style Trends by Location

In [ ]:
#Extracting the most popular (in quantity) beer style for each location by looking at the total number of ratings by year. We also put a threshold of 5 that needs to be satisfied for a location-style pair to be considered in this analysis.

years = np.arange(2006, 2018)
style_popularities_by_year = pd.DataFrame()
for year in years:
    location_style = location_style_stats(ratings.copy(), users.copy(), year, 5)
    location_style_groupedby_loc = location_style.groupby('location')
    idx_most_popular = location_style_groupedby_loc['number'].idxmax()
    most_popular_styles_by_location = location_style.loc[idx_most_popular]
    most_popular_styles_by_location = most_popular_styles_by_location[['location', 'style']]
    most_popular_styles_by_location.set_index('location', inplace = True)
    most_popular_styles_by_location = most_popular_styles_by_location.rename(columns = {'style': str(year)})
    style_popularities_by_year = pd.concat([style_popularities_by_year, most_popular_styles_by_location], axis = 1)

style_popularities_by_year

In [ ]:
#Extracting the highest rated beer style for each location by looking at the average rating by year. We also put a threshold of 5 that needs to be satisfied for a location-style pair to be considered in this analysis.

years = np.arange(2006,2018)

style_ratings_by_year = pd.DataFrame()
for year in years:
    location_style = location_style_stats(ratings.copy(), users.copy(), year, 5)
    location_style_groupedby_loc = location_style.groupby('location')
    idx_highest_rated = location_style_groupedby_loc['z_score'].idxmax()
    highest_rated_styles_by_location = location_style.loc[idx_highest_rated]
    highest_rated_styles_by_location = highest_rated_styles_by_location[['location', 'style']]
    highest_rated_styles_by_location.set_index('location', inplace = True)
    highest_rated_styles_by_location = highest_rated_styles_by_location.rename(columns = {'style': str(year)})
    style_ratings_by_year = pd.concat([style_ratings_by_year, highest_rated_styles_by_location], axis = 1)

style_ratings_by_year

3.2.2: Initial Analysis of Brewery Trends by Location

In [ ]:
#Extracting the most popular (in quantity) beer production place (brewery location) for each location by looking at the total number of ratings. We also put a threshold of 5 that needs to be satisfied for a location-brewery location pair to be considered in this analysis.

years = np.arange(2006,2018)
bc_popularities_by_year = pd.DataFrame()

for year in years:
    location_bc = location_brewery_country_stats(ratings.copy(), users.copy(), breweries.copy(), year, 5)
    location_bc_gb_loc = location_bc.groupby('location')
    idx_most_popular = location_bc_gb_loc['number'].idxmax()
    most_popular_bc_by_location = location_bc.loc[idx_most_popular]
    most_popular_bc_by_location = most_popular_bc_by_location[['location', 'brewery_location']]
    most_popular_bc_by_location.set_index('location', inplace = True)
    most_popular_bc_by_location = most_popular_bc_by_location.rename(columns = {'brewery_location': str(year)})
    bc_popularities_by_year = pd.concat([bc_popularities_by_year, most_popular_bc_by_location], axis = 1)
    
bc_popularities_by_year

In [ ]:
#Extracting the highest rated beer production location for each location by looking at the average rating by year. We also put a threshold of 10 that needs to be satisfied for a location-brewery location pair to be considered in this analysis.

years = np.arange(2006, 2018)
bc_ratings_by_year = pd.DataFrame()

for year in years:
    location_bc = location_brewery_country_stats(ratings.copy(), users.copy(), breweries.copy(), year, 10)
    location_bc_gb_loc = location_bc.groupby('location')
    idx_highest_rated = location_bc_gb_loc['z_score'].idxmax()
    highest_rated_bc_by_location = location_bc.loc[idx_highest_rated]
    highest_rated_bc_by_location = highest_rated_bc_by_location[['location', 'brewery_location']]
    highest_rated_bc_by_location.set_index('location', inplace=True)
    highest_rated_bc_by_location = highest_rated_bc_by_location.rename(columns={'brewery_location': str(year)})
    bc_ratings_by_year = pd.concat([bc_ratings_by_year, highest_rated_bc_by_location], axis=1)
    
bc_ratings_by_year

3. Example: Distance Calculations

In [ ]:
ll_dict = dict(zip(all_locations, location_latitudes_longitudes_tuples))
ll_dict

In [ ]:
#Printing examples of distances between three location pairs using geodesic() function:

print(f"Distance between New York and Ohio is {geodesic(ll_dict['United States, New York'], ll_dict['United States, Ohio']).kilometers:.2f} kilometers.")

print(f"Distance between Brazil and Germany is {geodesic(ll_dict['Brazil'], ll_dict['Germany']).kilometers:.2f} kilometers.")

print(f"Distance between Iceland and New Zealand is {geodesic(ll_dict['Iceland'], ll_dict['New Zealand']).kilometers:.2f} kilometers.")

4. Initial Regression and Correlation Analysis

We study the correlation between the different aspects of the beers, and their rating.

In [ ]:
#Linear regression of the rating suing the features "appearance", "aroma", "palate" and "taste".

mod = smf.ols(formula = 'rating ~ appearance + aroma + palate + taste', data = ratings)
np.random.seed(2)
res = mod.fit()
print(res.summary())

The results suggest that the taste property is the most important and the appearance property is the least important factor for a beer to receive a higher rating. Furthermore, p-values for all the coefficients are 0.000 and explained variance (R-squared) is 0.962. This shows that the rating is highly correlated and explainable with the given features, as expected.

In [ ]:
corr, p_value = stats.pearsonr(ratings['abv'], ratings['rating'])

print(f"Pearson correlation between alcohol by volume and rating is {corr:.3f} with a p value of {p_value:.3f}.")

We see that alcohol by volume is significantly correlated with higher ratings, with a pearson correlation coefficient of 0.350. Furthermore, p_value is given as 0.000, meaning that having no actual correlation between them but still observing the data we have is almost impossible. This is a rather surprising and remarkable finding to have in mind for our future analysis.